In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_columns', None) # 모든 열 표시

## 예술의 전당 전처리

In [3]:
preprocessed = pd.read_csv('../preprocessed_data/예술의 전당/예술의전당.csv')
corona = pd.read_csv('../corona_index/서울_코로나지수_최종.csv',encoding = 'cp949')
preprocessed = preprocessed.drop_duplicates(subset='공연코드')

코로나 표준점수 컬럼 추가

In [3]:
corona['날짜'] = pd.to_datetime(corona['날짜'])
# '공연시작일자'에서 년, 월, 일 추출
corona['년'] = corona['날짜'].dt.year
corona['월'] = corona['날짜'].dt.month
corona['일'] = corona['날짜'].dt.day

preprocessed['코로나표준점수']=0

for i in range(len(preprocessed)):
    year = preprocessed.iloc[i,:]['공연시작년도']
    month = preprocessed.iloc[i,:]['공연시작월']
    day = preprocessed.iloc[i,:]['공연시작일']
    filtered_row = corona.loc[(corona['년'] == year) & (corona['월'] == month) & (corona['일'] == day)]
    if len(filtered_row) ==1:
        preprocessed['코로나표준점수'].iloc[i]=filtered_row['표준화단계'].iloc[0]
    else:
        pass

C:\Users\passion\AppData\Local\Temp\ipykernel_11236\1512425699.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed['코로나표준점수'].iloc[i]=filtered_row['표준화단계'].iloc[0]


trend index추가 

In [30]:
dirc = r'../trend_index/예술의전당'
ext = r'.csv'

trend_index_files = [file for file in os.listdir(dirc) if file.endswith(ext)] #여기에는 출연진 csv파일만 있음

In [5]:
aug_df = pd.DataFrame()
for f in trend_index_files:
    file_path = os.path.join(dirc, f)
    try:
        df = pd.read_csv(file_path, encoding='cp949')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='utf-8')
    
    aug_df = pd.concat([aug_df, df])
# '성별' 컬럼 제거
aug_df = aug_df.drop(columns=['출연진내용'])

In [6]:
preprocessed.shape

(1006, 26)

In [7]:
result_df = pd.merge(preprocessed, aug_df, on='공연코드')
result_df.shape

(1006, 28)

In [8]:
result_df.to_csv('../preprocessed_data/예술의전당/예술의전당_preprocessed.csv',index = False)

In [9]:
result_df

,Unnamed: 0,공연코드,공연회차,소요시간,장르명,세부장르명,출연진내용,제작진내용,기획제작사명,관람연령,...,공연시작요일,공연시작시분,공연기간,공휴일여부,좌석등급개수,평균티켓가격,좌석점유율,코로나표준점수,검색량,출연횟수
0,SE000201,PF372853,1,110,서양음악(클래식),기악,"김주원, 쾅 하오 후앙",NaN,(주)마스트미디어(주최),만 7세 이상,...,Tuesday,1200,1,0,1,30000.0,0.460452,0.0,30,11
1,SE000201,PF360668,1,100,서양음악(클래식),기악,"최유진, 이재완",NaN,예인예술기획(주최),만 7세 이상,...,Wednesday,1200,1,0,1,20000.0,0.968927,0.0,0,10
2,SE000201,PF372854,1,100,서양음악(클래식),기악,김민욱,NaN,가온클래식(주최),만 7세 이상,...,Thursday,1200,1,0,1,20000.0,0.742938,0.0,0,1
3,SE000201,PF360799,1,80,서양음악(클래식),기악,"김경희, 신미정, 장혜리, 안동호",NaN,세광아트(세광음악출판사)(주관),만 7세 이상,...,Friday,1200,1,0,1,30000.0,0.946541,0.0,220,10
4,SE000201,PF372069,1,100,서양음악(클래식),기악,나윤아,NaN,마에스트로 컴퍼니(주최),만 7세 이상,...,Saturday,1200,1,0,1,10000.0,0.593525,0.0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,SE000201,PF424244,1,90,서양음악(클래식),기악,"유의정, 김정현, 윤효린",NaN,지클레프(주최),만 7세 이상,...,Wednesday,1170,1,0,1,20000.0,0.946328,0.0,0,10
1002,SE000101,PF426471,1,90,서양음악(클래식),기악,이주용,NaN,NaN,만 7세 이상,...,Thursday,1170,1,0,1,30000.0,0.728261,0.0,0,19
1003,SE000201,PF425379,1,90,서양음악(클래식),기악,"김길우, 정태민",NaN,지클레프(주최),만 7세 이상,...,Friday,1170,1,0,1,20000.0,0.951977,0.0,0,2
1004,SE000201,PF424637,1,90,서양음악(클래식),기악,박요셉,NaN,지클레프(주최),만 7세 이상,...,Saturday,840,1,0,2,40000.0,0.954802,0.0,0,5


## 대구콘서트하우스 전처리

In [20]:
preprocessed = pd.read_csv('../preprocessed_data/대구콘서트하우스/대구콘서트하우스.csv')
corona = pd.read_csv('../corona_index/대구_코로나지수_최종.csv',encoding = 'cp949')

코로나 표준점수 컬럼 추가

In [21]:
corona['날짜'] = pd.to_datetime(corona['날짜'])
# '공연시작일자'에서 년, 월, 일 추출
corona['년'] = corona['날짜'].dt.year
corona['월'] = corona['날짜'].dt.month
corona['일'] = corona['날짜'].dt.day

preprocessed['코로나표준점수']=0

for i in range(len(preprocessed)):
    year = preprocessed.iloc[i,:]['공연시작년도']
    month = preprocessed.iloc[i,:]['공연시작월']
    day = preprocessed.iloc[i,:]['공연시작일']
    filtered_row = corona.loc[(corona['년'] == year) & (corona['월'] == month) & (corona['일'] == day)]
    if len(filtered_row) ==1:
        preprocessed['코로나표준점수'].iloc[i]=filtered_row['표준화단계'].iloc[0]
    else:
        pass

C:\Users\passion\AppData\Local\Temp\ipykernel_19816\1512425699.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed['코로나표준점수'].iloc[i]=filtered_row['표준화단계'].iloc[0]


trend index추가 

In [22]:
dirc = r'../trend_index/대구콘서트하우스'
ext = r'.csv'

trend_index_files = [file for file in os.listdir(dirc) if file.endswith(ext)] #여기에는 출연진 csv파일만 있음

In [23]:
aug_df = pd.DataFrame()
for f in trend_index_files:
    file_path = os.path.join(dirc, f)
    try:
        df = pd.read_csv(file_path, encoding='cp949')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='utf-8')
    
    aug_df = pd.concat([aug_df, df])
# '성별' 컬럼 제거
aug_df = aug_df.drop(columns=['출연진내용'])

In [24]:
preprocessed.shape

(310, 27)

In [25]:
result_df = pd.merge(preprocessed, aug_df, on='공연코드')
result_df.shape

(310, 29)

In [29]:
result_df.to_csv('../preprocessed_data/대구콘서트하우스/대구콘서트하우스_preprocessed.csv',index = False)

In [27]:
result_df

,Unnamed: 0,전송사업자코드,공연코드,공연회차,소요시간,장르명,세부장르명,출연진내용,제작진내용,기획제작사명,...,공연시작요일,공연시작시분,공연기간,공휴일여부,좌석등급개수,평균티켓가격,좌석점유율,코로나표준점수,검색량,출연횟수
0,0,SE000102,PF360097,1,90,서양음악(클래식),기악,한성원,NaN,대구콘서트하우스(주최),...,Tuesday,1170,1,0,1,10000.0,0.995968,0,0,7
1,1,SE000102,PF360828,1,70,서양음악(클래식),기악,"최민정, 배규희, 우현수, 정혜진, 남자은",NaN,대구콘서트하우스(주최),...,Tuesday,1170,1,0,1,10000.0,0.983871,0,0,4
2,2,SE000303,PF373182,1,60,서양음악(클래식),기악,"김은지, 오은비, 형성경, 신유리",NaN,"블랑앳누아(주최), 블랑앳누아(주관)",...,Saturday,1170,1,0,1,10000.0,0.024194,0,0,2
3,3,SE000102,PF360028,1,70,서양음악(클래식),성악,"권재희, 정지은",NaN,대구콘서트하우스(주최),...,Friday,1170,1,0,1,10000.0,0.754032,0,0,2
4,4,SE000102,PF373483,1,90,서양음악(클래식),기악,주석용,NaN,"대구콘서트하우스(주최), 대구콘서트하우스(기획사)",...,Friday,1170,1,0,1,10000.0,0.991935,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,305,SE000102,PF425546,1,80,서양음악(클래식),성악,"이정아, 배혜리, 이영규, 백민아, 구은정, 최민영, 김아름 등",NaN,"더 소프라노즈(주최), 더 소프라노즈(주관)",...,Wednesday,1170,1,0,1,20000.0,1.362903,0,0,10
306,306,SE000201,PF424911,1,90,서양음악(클래식),기악,"이호찬, 박상욱",NaN,"대구콘서트하우스(주최), 대구콘서트하우스(주관)",...,Friday,1170,1,0,1,30000.0,0.596774,0,520,14
307,307,SE000201,PF424030,1,90,대중음악,재즈/월드뮤직,"김우연, 김재원, 김종호, 이찬희, 고상지",NaN,"대구콘서트하우스(주최), 대구콘서트하우스(주관)",...,Tuesday,1170,1,0,1,10000.0,0.959677,0,1930,155
308,308,SE000303,PF425801,1,70,서양음악(클래식),기악,김혜령,NaN,NaN,...,Thursday,1170,1,0,1,10000.0,1.000000,0,0,9
